# Model Risk MVP

_Initial commit: Anton Markov, 1 October 2021_

_Latest edit: Anton Markov, 2 October 2021_

Основная цель данного ноутбука — построить базовую структуру модели кредитного 
скоринга. Модульную архитектуру реализую потом. 

__Логика модели:__

1. Импорт данных. Разбивка данных на TRAIN / TEST
3. Missing values imputation
4. Preprocessing
5. Baseline
6. Feature selection
7. Hyperopt
8. Final model training
9. Model validation


In [ ]:
import time
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets, metrics, model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

from hyperopt import hp
# from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# for HyperOpt class
import lightgbm as lgb
import xgboost as xgb
# import catboost as ctb
from hyperopt import fmin, tpe, STATUS_OK, STATUS_FAIL, Trials

In [ ]:
# params
seed = 42
dataset = 'german'
num_factors = 10

In [ ]:
# сохраним окружение
!pip freeze > /content/requirements.txt
# !conda env export > /content/requirements.yml

In [ ]:
# засекаем время
startTime = time.time()

## 1. Импорт данных. Разбивка TRAIN / TEST

Пока подгружаю и предобрабатываю в одном пайплайне, по-хорошему этот блок лучше изолировать: у каждого датасета своя атмосфера, эту часть сложно унифицировать, в модульной архитектуре это будет мешать.

Визуальный анализ не делаю, его следует делать по каждому датасету отдельно. Реализуем, когда будет модульная архитектура.

### 1.1. German Dataset

[Ссылка на скачивание](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data))

Самый популярный, но довольно маленький датасет. Подгружаю только из-за отсутствия необходимости делать какие-то преобразования: многое уже подчищено.

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data',
                 header = None, sep = ' ')

# based on the .doc data description
df.columns = ['cheq_acc', 'dur_t', 'cred_hist', 'purp', 'cred_amt', 'save_acc', 
              'empl_t', 'inst_to_income', 'pers_status', 'guarant_flg',
              'residence_t', 'prop', 'age', 'inst_plan', 'house', 'n_loans',
              'job', 'n_depend', 'tel_flg', 'foreign_flg', 'target']

df.head()

,cheq_acc,dur_t,cred_hist,purp,cred_amt,save_acc,empl_t,inst_to_income,pers_status,guarant_flg,residence_t,prop,age,inst_plan,house,n_loans,job,n_depend,tel_flg,foreign_flg,target
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,4,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,2,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,3,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,4,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,4,A124,53,A143,A153,2,A173,2,A191,A201,2


Очевидно, что мы имеем большое число категориальных переменных. В дальнейшем мы заменим их при помощи WOE-преобразования. На данный момент лишь сформируем разбивку train vs. test и запишем список переменных

In [ ]:
X = df.loc[:, df.columns != 'target']
y = df.target - 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

# Save data & info ===
# parquet is optimized for large volumes of data
!mkdir /content/german
X_train.to_parquet('/content/german/X_train.parquet')
X_test.to_parquet('/content/german/X_test.parquet')
# переводим pd.Series в pd.DataFrame для удобного экспорта
pd.DataFrame(y_train).to_parquet('/content/german/y_train.parquet')
pd.DataFrame(y_test).to_parquet('/content/german/y_test.parquet')

# сохраняем списки категориальных и колич. переменных
cat_vals = ['cheq_acc', 'cred_hist', 'purp', 'save_acc', 
            'empl_t', 'pers_status', 'guarant_flg', 'prop', 
            'inst_plan', 'house', 'job', 'tel_flg', 'foreign_flg']
num_vals = ['dur_t', 'cred_amt', 'inst_to_income', 'residence_t', 
            'age', 'n_loans', 'n_depend']

with open('/content/german/factors.json', 'w') as f:
    json.dump({'cat_vals': cat_vals, "num_vals": num_vals}, f)

### 1.2. Taiwanese Dataset

[Ссылка на скачивание](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)

Достаточно большой датасет, который также встречается во многих статьях.

In [ ]:
df = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls', skiprows=1)
df.drop('ID', axis='columns', inplace=True)
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


Судя по всему, все переменные количественные. Нашим легче.

In [ ]:
X = df.loc[:, df.columns != 'default payment next month']
y = df['default payment next month']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

# Save data & info ===
# parquet is optimized for large volumes of data
!mkdir /content/taiwanese
X_train.to_parquet('/content/taiwanese/X_train.parquet')
X_test.to_parquet('/content/taiwanese/X_test.parquet')
# переводим pd.Series в pd.DataFrame для удобного экспорта
pd.DataFrame(y_train).to_parquet('/content/taiwanese/y_train.parquet')
pd.DataFrame(y_test).to_parquet('/content/taiwanese/y_test.parquet')

# сохраняем списки категориальных и колич. переменных
cat_vals = []
num_vals = df.columns.to_list()

with open('/content/taiwanese/factors.json', 'w') as f:
    json.dump({'cat_vals': cat_vals, "num_vals": num_vals}, f)

## 2. Missing Values Imputation

В первую очередь мы подгружаем датасет, указанный в параметрах


In [ ]:
X_train = pd.read_parquet(f'/content/{dataset}/X_train.parquet')
X_test = pd.read_parquet(f'/content/{dataset}/X_test.parquet')

# переводим обратно в pd.Series
y_train = pd.read_parquet(f'/content/{dataset}/y_train.parquet').iloc[:, 0]
y_test = pd.read_parquet(f'/content/{dataset}/y_test.parquet').iloc[:, 0]

# испортируем типы переменных
with open(f'/content/{dataset}/factors.json', 'r') as f:
    vals_dict = json.load(f)
cat_vals = vals_dict['cat_vals']
num_vals = vals_dict['num_vals']


Поскольку в обоих рассматриваемых датасетах все факторы заполнены, мы пока ставим "заглушку".

In [ ]:
X_train.isna().sum(axis=0)

cheq_acc          0
dur_t             0
cred_hist         0
purp              0
cred_amt          0
save_acc          0
empl_t            0
inst_to_income    0
pers_status       0
guarant_flg       0
residence_t       0
prop              0
age               0
inst_plan         0
house             0
n_loans           0
job               0
n_depend          0
tel_flg           0
foreign_flg       0
dtype: int64

In [ ]:
X_test.isna().sum(axis=0)

cheq_acc          0
dur_t             0
cred_hist         0
purp              0
cred_amt          0
save_acc          0
empl_t            0
inst_to_income    0
pers_status       0
guarant_flg       0
residence_t       0
prop              0
age               0
inst_plan         0
house             0
n_loans           0
job               0
n_depend          0
tel_flg           0
foreign_flg       0
dtype: int64

In [ ]:
X_train.fillna(999, inplace=True)
X_test.fillna(999, inplace=True)

## 3. Preprocessing

Теоретически здесь должна быть ребалансировка или feature engineering, но пока этот раздел опустим. В обоих датасетах 20-30% целевых событий, что не так плохо изначально. Поэтому пока сделаю только WOE-трансформацию для категориальных переменных.

### 3.1. Ребалансировка

### 3.2. WOE-трансформация

In [ ]:
# технические детали, могу устно пояснить — Антон

# готовим библиотеку через одно место: скачиваем только 3 нужных скрипта
!git clone https://github.com/sberbank-ai/wing /content/wing
!cp /content/wing/wing/core/main.py /content/main.py 
!cp /content/wing/wing/core/optimizer.py /content/optimizer.py 
!cp /content/wing/wing/core/functions.py /content/functions.py 
!rm -R /content/wing

# заменяем относительный reference на абсолютный
!sed -i 's/.functions/functions/g' /content/optimizer.py
!sed -i 's/.functions/functions/g' /content/main.py
!sed -i 's/.optimizer/optimizer/g' /content/main.py

# косяк в изначальной либе
!sed -i 's/selfoptimizer/self.optimizer/g' /content/optimizer.py
!sed -i 's/selfoptimizer/self.optimizer/g' /content/main.py

Cloning into '/content/wing'...
remote: Enumerating objects: 62, done.
remote: Total 62 (delta 0), reused 0 (delta 0), pack-reused 62
Unpacking objects: 100% (62/62), done.


In [ ]:
from main import WingsOfEvidence

In [ ]:
X_test.head(3)

,cheq_acc,dur_t,cred_hist,purp,cred_amt,save_acc,empl_t,inst_to_income,pers_status,guarant_flg,residence_t,prop,age,inst_plan,house,n_loans,job,n_depend,tel_flg,foreign_flg
0,A11,18,A32,A43,3190,A61,A73,2,A92,A101,2,A121,24,A143,A152,1,A173,1,A191,A201
1,A11,18,A32,A40,4380,A62,A73,3,A93,A101,4,A123,35,A143,A152,1,A172,2,A192,A201
2,A11,24,A31,A40,2325,A62,A74,2,A93,A101,3,A123,32,A141,A152,1,A173,1,A191,A201


In [ ]:
##########################################################################
### ВНИМАНИЕ! ### ЕСЛИ КТО-ТО УМЕЕТ УКАЗЫВАТЬ СИД В WOE, НАУЧИТЕ МЕНЯ! ###
##########################################################################
wings = WingsOfEvidence(
    columns_to_apply=cat_vals, 
    n_initial=20, 
    n_target=10, 
    only_values=True
)
wings.fit(X_train, y_train)

X_train_WOE = wings.transform(X_train)
X_test_WOE =  wings.transform(X_test)

X_train_WOE.to_parquet(f'/content/{dataset}/X_train_WOE.parquet')
X_test_WOE.to_parquet(f'/content/{dataset}/X_test_WOE.parquet')

for c in cat_vals:
    X_train[c] = X_train_WOE['WOE_' + c]
    X_test[c]  = X_test_WOE['WOE_' + c]

In [ ]:
X_test.head(3)

,cheq_acc,dur_t,cred_hist,purp,cred_amt,save_acc,empl_t,inst_to_income,pers_status,guarant_flg,residence_t,prop,age,inst_plan,house,n_loans,job,n_depend,tel_flg,foreign_flg
0,-0.788869,18,-0.160963,0.513722,3190,-0.231914,-0.029193,2,-0.280309,-0.021201,2,0.544924,24,0.090352,0.218116,1,0.012056,1,-0.066374,0.0
1,-0.788869,18,-0.160963,-0.433830,4380,-0.182016,-0.029193,3,0.179744,-0.021201,4,-0.043180,35,0.090352,0.218116,1,0.133837,2,0.093026,0.0
2,-0.788869,24,-1.247152,-0.433830,2325,-0.182016,0.101402,2,0.179744,-0.021201,3,-0.043180,32,-0.381505,0.218116,1,0.012056,1,-0.066374,0.0


### 3.3. Feature Engineering

### 3.4. Исключение нестабильных во времени переменных

Надо, если данные по месяцам, а не как у нас.

## 4. Baseline

Подберем бейзлайн из списка

* LightGBM
* XGBoost 
* Random Forest
* Decision Tree

CatBoost в Colab'e не установлен, добавим позже.

In [ ]:
def calc_ginis(mdl, X_list, y_list, label_list):
    # на каждой из паре выборка - таргет (X, y) измеряет
    # и принтит Gini
    ginis = {}
    for i in range(len(X_list)):
        y_pred = mdl.predict_proba(X_list[i])
        gini = roc_auc_score(y_list[i], y_pred[:, 1]) * 2 - 1
        print(f'GINI {label_list[i]}: {gini}')
        ginis[label_list[i]] = gini
    return(ginis)

### 4.1. LightGBM

In [ ]:
%%time
lgbm_mdl = LGBMClassifier(
    num_leaves = 10,
    learning_rate = .1,
    reg_alpha = 8,
    reg_lambda = 8,
    random_state = seed
)
lgbm_mdl.fit(X_train, y_train, verbose = True)

CPU times: user 48.8 ms, sys: 6.17 ms, total: 55 ms
Wall time: 43.2 ms


In [ ]:
calc_ginis(
    mdl = lgbm_mdl,
    X_list = [X_train, X_test],
    y_list = [y_train, y_test],
    label_list = ['TRAIN', 'TEST']
)

GINI TRAIN: 0.7259864157709583
GINI TEST: 0.5355171144644828


{'TEST': 0.5355171144644828, 'TRAIN': 0.7259864157709583}

Жесткое переобучение! Будем разбираться...

### 4.2. XGBoost

In [ ]:
%%time
xgb_mdl = XGBClassifier(
    max_depth = 3,
    learning_rate = .1,
    reg_alpha = 8,
    reg_lambda = 8,
    seed = seed
)
xgb_mdl.fit(X_train, y_train, verbose = True)

CPU times: user 81 ms, sys: 17.6 ms, total: 98.6 ms
Wall time: 215 ms


In [ ]:
calc_ginis(
    mdl = xgb_mdl,
    X_list = [X_train, X_test],
    y_list = [y_train, y_test],
    label_list = ['TRAIN', 'TEST']
)

GINI TRAIN: 0.7356629863865365
GINI TEST: 0.5462432304537568


{'TEST': 0.5462432304537568, 'TRAIN': 0.7356629863865365}

Переобучение ещё чуть больше, но качетсов на тесте тоже выше.

### 4.3. Random Forest

In [ ]:
%%time
rf_mdl = RandomForestClassifier(
    max_depth = 3,
    random_state = seed,
    verbose=0
)
rf_mdl.fit(X_train, y_train)

CPU times: user 136 ms, sys: 991 µs, total: 137 ms
Wall time: 141 ms


In [ ]:
calc_ginis(
    mdl = rf_mdl,
    X_list = [X_train, X_test],
    y_list = [y_train, y_test],
    label_list = ['TRAIN', 'TEST']
)

GINI TRAIN: 0.7028425535232268
GINI TEST: 0.581471160418529


{'TEST': 0.581471160418529, 'TRAIN': 0.7028425535232268}

Ого! RF лучше бустингов! Что-то тут не так... Надо очень сильно в регуляризацию.

### 4.4. Decision Tree

In [ ]:
%%time
tree_mdl = DecisionTreeClassifier(
    max_leaf_nodes = 10,
    random_state = seed
)
tree_mdl.fit(X_train, y_train)

CPU times: user 3.84 ms, sys: 0 ns, total: 3.84 ms
Wall time: 3.98 ms


In [ ]:
calc_ginis(
    mdl = tree_mdl,
    X_list = [X_train, X_test],
    y_list = [y_train, y_test],
    label_list = ['TRAIN', 'TEST']
)

GINI TRAIN: 0.5710346037283545
GINI TEST: 0.5004469214995533


{'TEST': 0.5004469214995533, 'TRAIN': 0.5710346037283545}

Не лучше и не хуже бустингов, но без переобучения.

### 4.5. Conclusion

Пока кажется, что бустинги и RF — лучшие алгоритмы. Сам я не могу поверить, что RF лучше бустингов, поэтому будем работать над регуляризацией.

## 5. Feature Selection

Предлагаю in the long run задать # переменных как экзогенный фактор при оптимизации, пока задаю экспертно (от фонаря).

In [ ]:
# reminder
print(f'В параметрах задано следующее число факторов в финальной модели: {num_factors}')

В параметрах задано следующее число факторов в финальной модели: 10
